In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/'
!ls

/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR
CrossValidationScore.py		  Predictions.py
cv_split.pickle			  predict.py
data				  __pycache__
EDA.ipynb			  README.md
initial_test_b3_fold3_clahe	  split_folds.ipynb
logs				  submit
MNIST				  submit_initial_test_b3.csv
model_concate_10_no_epoch.txt	  submit_oanh_initial_test_b3.csv
model_concate_15epoch_weight.txt  submit_vessel_initial_test_b3.csv
model_concate_19epoch.txt	  train_1cycle.py
model_concate_20epoch.txt	  train_all_folds_args.sh
model_concate_5_no_epoch.txt	  train_all_folds.sh
model_concate_6epoch.txt	  train.py
model_concate_epoch_20.txt	  utils.py
odir_submit.py			  Visualization.ipynb


In [ ]:
!pip install catalyst==20.2.4
!pip install tqdm==4.33
!pip install efficientnet_pytorch
!pip install pytorch_toolbelt

In [ ]:
from sklearn import svm
from keras.callbacks import CSVLogger
from keras.callbacks import LambdaCallback
from utils import *
#from simple_model import *
#from preprocess import *
#from make_labels import *
import pandas as pd
import sys
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy as np
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import torch
import collections
from pytorch_toolbelt.inference import tta
from catalyst.dl.callbacks import InferCallback
from catalyst.dl.runner import SupervisedRunner
from torch.nn.functional import softmax
from catalyst.dl.callbacks import EarlyStoppingCallback, AccuracyCallback, F1ScoreCallback, ConfusionMatrixCallback, MixupCallback
from catalyst.contrib.nn.schedulers import OneCycleLR, ReduceLROnPlateau, StepLR, MultiStepLR
from torchvision import transforms
import torch.nn.functional as F
from torch.autograd import Variable
from torchsummary import summary
import pickle
import time
import copy


Using TensorFlow backend.
alchemy not available, to install alchemy, run `pip install alchemy-catalyst`.


**prepare data**

In [ ]:
data = pd.read_csv(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/splited_train.csv')
splits = pickle.load(open(
    '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/cv_split.pickle', 'rb'))
labels = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
image_size = 240
fold_idx = 3
batch_size = 1
lr = 1e-5

In [ ]:
train_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/ODIR-5K_Training_Dataset/'
valid_path1 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/ODIR-5K_Training_Dataset/'

train_dataset1 = EyeDataset(dataset_path=train_path1,
                           labels=data.loc[splits['train_idx'][fold_idx],labels].values,
                           ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))

# val_dataset1 = EyeDataset(dataset_path=valid_path1, 
# 					                labels=data.loc[splits['test_idx'][fold_idx],labels].values, 
# 					                ids=data.loc[splits['test_idx'][fold_idx],'id'].values, 
# 					                albumentations_tr=aug_val(image_size))
# train_dataset1 = EyeDataset(dataset_path=train_path1,
#                            labels=data.loc[:,labels].values,
#                            ids=data.loc[:, 'id'].values,
#                            albumentations_tr=aug_train_heavy(image_size))

train_loader1 = DataLoader(train_dataset1,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)


logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/initial_test_b3_fold3_org/'
modelA = prepare_model('efficientnet-b3', 8)
modelA.cuda()
modelA.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/initial_test_b3_fold3_org/checkpoints/best.pth'))['model_state_dict'])
modelA.eval()

In [ ]:
train_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/training-dataset/'
valid_path2 = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/data/training-dataset/'

# train_dataset2 = EyeDataset(dataset_path=train_path2,
#                            labels=data.loc[splits['train_idx'][fold_idx], labels].values,
#                            ids=data.loc[splits['train_idx'][fold_idx], 'id'].values,
#                            albumentations_tr=aug_train_heavy(image_size))
# val_dataset2 = EyeDataset(dataset_path=valid_path2, 
# 					                labels=data.loc[splits['test_idx'][fold_idx],labels].values, 
# 					                ids=data.loc[splits['test_idx'][fold_idx],'id'].values, 
# 					                albumentations_tr=aug_val(image_size))
train_dataset2 = EyeDataset(dataset_path=train_path2,
                           labels=data.loc[:, labels].values,
                           ids=data.loc[:, 'id'].values,
                           albumentations_tr=aug_train_heavy(image_size))

train_loader2 = DataLoader(train_dataset2,
                          num_workers=8,
                          pin_memory=False,
                          batch_size=batch_size,
                          shuffle=False)

logdir = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/initial_test_b3_fold3_vessel/'
modelB = prepare_model('efficientnet-b3', 8)
modelB.cuda()
modelB.load_state_dict(torch.load(os.path.join(
    logdir, '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/initial_test_b3_fold3_vessel/checkpoints/best.pth'))['model_state_dict'])
modelB.eval()

In [ ]:
for i1 in train_loader1:
  input1,target1 = i1
  #print(input1)
  #print(target1)
  print(i1)
  input1 = input1.cuda()
  break


[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 

In [ ]:
for i2 in train_loader2:
  input2,target2 = i2
  #print(input2)
  #print(target2)
  input2 = input2.cuda()
  print(i2)
  break  

[tensor([[[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],

         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],

         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044, 


**FORGET GATE 2 MODEL**

In [ ]:
import sklearn
class NeuralNet(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=8):
        super(NeuralNet, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        self.drop = nn.Dropout(0.3)
        self.out2 = nn.Linear(1536, 8)
        self.modelA.fc = nn.Identity()
        self.modelB.fc = nn.Identity()

    def forward(self, x1, x2):
        x1 = self.modelA.extract_features(x1)
        x2 = self.modelB.extract_features(x2)
        
        x1 = nn.AdaptiveAvgPool2d((1, 1))(x1)
        x2 = nn.AdaptiveAvgPool2d((1, 1))(x2)
        x1 = x1.view(1,1536)
        x1 = torch.Tensor.cpu(x1).detach().numpy()
        x2 = x2.view(1,1536)
        x2 = torch.Tensor.cpu(x2).detach().numpy()
        x = x1+x2        
        sigmoid = tf.math.sigmoid(x)
        multi = tf.math.multiply(sigmoid,x2)
        add = multi + x1
        add = add.numpy()
        x = torch.from_numpy(add)
        x = self.drop(x)
        x = x.flatten()
        x = self.out2(x)
        return x

In [ ]:
probabilities = []
x_model = NeuralNet(modelA, modelB)
y = x_model(input1,input2)
print(y)
print(type(y))
y = torch.Tensor.cpu(y).detach().numpy()
print(y)
print(type(y))
probabilities = softmax(torch.from_numpy(y),dim=0).numpy()
print(probabilities)

tensor([ 0.1252,  0.0514, -0.0458, -0.1257,  0.1552,  0.0167,  0.0959, -0.2829],
       grad_fn=<AddBackward0>)
<class 'torch.Tensor'>
[ 0.12519808  0.05135238 -0.04580437 -0.12567937  0.15521011  0.01673217
  0.09585901 -0.2829435 ]
<class 'numpy.ndarray'>
[0.14057976 0.13057259 0.11848336 0.1093876  0.14486279 0.1261295
 0.13651519 0.09346933]


In [ ]:
x_model = NeuralNet(modelA, modelB)


In [ ]:
x_model.load_state_dict(torch.load(os.path.join(
        '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/concate/model_concate_epoch5.pth')))
x_model.eval()

In [ ]:

#optimizer = torch.optim.Adam(x_model.parameters(), lr=1e-5)
optimizer = torch.optim.Adam(x_model.parameters(), lr=3e-4)
#weights = torch.tensor([0.5, 0.6, 0.7, 0.7, 0.7, 0.7,0.7, 0.6])
#weights = weights.cuda()
#criterion = nn.CrossEntropyLoss(reduction='none')
criterion = nn.CrossEntropyLoss()
#scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.75, patience=2)
scheduler = ReduceLROnPlateau(optimizer=optimizer, factor=0.5, patience=5)

dem = 0
loss_log = []
PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/concate_fromepoch5_lr/'
if not os.path.exists(PATH_SAVE):
       os.makedirs(PATH_SAVE)
for epoch in range(20):
  print('====Epoch ',epoch,": ")
  for i1, i2 in zip(enumerate(train_loader1),enumerate(train_loader2)):
    dem = dem + 1
    #print(i1)
    stt1, data1 = i1
    input1,target1 = data1
    #print(target1)
    #print(i2)
    stt2, data2 = i2
    input2,target2 = data2
    #print(target2)
    # zero the parameter gradients
    optimizer.zero_grad()
    input1 = input1.cuda()
    input2 = input2.cuda()

    # forward + backward + optimize
    outputs = x_model(input1,input2)
    #print(outputs)
    #print(target1)
    outputs = outputs.view(1,8)
    loss = criterion(outputs, target1)
    loss.backward()
    optimizer.step()
    if dem%1000==0:
        print('so luong: ',dem,'/',len(train_loader1))
  loss_log.append(loss)
  print(loss)
  dem = 0
  torch.save(x_model.state_dict(), PATH_SAVE + 'model_concate_fromepoch5_lr_epoch_'+str(epoch)+'.pth')

with open(PATH_SAVE+"model_concate_fromepoch5_20epoch_lr.txt", "w") as output:
    output.write(str(loss_log))
    
#PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/concate/'
#if not os.path.exists(PATH_SAVE):
#        os.makedirs(PATH_SAVE)
#torch.save(x_model.state_dict(), PATH_SAVE + 'model_concate_30epoch.pth')

====Epoch  0 : 
so luong:  1000 / 6962
so luong:  2000 / 6962
so luong:  3000 / 6962
so luong:  4000 / 6962
so luong:  5000 / 6962
so luong:  6000 / 6962
tensor(11.0529, grad_fn=<NllLossBackward>)
====Epoch  1 : 
so luong:  1000 / 6962
so luong:  2000 / 6962
so luong:  3000 / 6962
so luong:  4000 / 6962
so luong:  5000 / 6962
so luong:  6000 / 6962
tensor(7.4027, grad_fn=<NllLossBackward>)
====Epoch  2 : 
so luong:  1000 / 6962
so luong:  2000 / 6962
so luong:  3000 / 6962
so luong:  4000 / 6962
so luong:  5000 / 6962
so luong:  6000 / 6962
tensor(8.6225, grad_fn=<NllLossBackward>)
====Epoch  3 : 
so luong:  1000 / 6962
so luong:  2000 / 6962
so luong:  3000 / 6962
so luong:  4000 / 6962
so luong:  5000 / 6962
so luong:  6000 / 6962
tensor(8.7669, grad_fn=<NllLossBackward>)
====Epoch  4 : 
so luong:  1000 / 6962
so luong:  2000 / 6962
so luong:  3000 / 6962
so luong:  4000 / 6962
so luong:  5000 / 6962
so luong:  6000 / 6962
tensor(9.2450, grad_fn=<NllLossBackward>)
====Epoch  5 : 
so 

In [ ]:
print(loss)

NameError: ignored

In [ ]:
with open("model_concate_19epoch.txt", "w") as output:
    output.write(str(loss_log))
    
PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/concate/'
if not os.path.exists(PATH_SAVE):
        os.makedirs(PATH_SAVE)
torch.save(x_model.state_dict(), PATH_SAVE + 'model_concate_19epoch.pth')

In [ ]:
PATH_SAVE = '/content/gdrive/My Drive/Colab Notebooks/odir-ivashnyov/ODIR/logs/concate/'
if not os.path.exists(PATH_SAVE):
        os.makedirs(PATH_SAVE)
torch.save(x_model.state_dict(), PATH_SAVE + 'model_concate_epoch10.pth')